# Nino box example

In [ ]:
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pylab as plt
import numpy as np
from matplotlib import cm
import pyresample

In [ ]:
%matplotlib inline

In [ ]:
opendap_data_url = 'http://esgf-data2.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/CMIP/MIROC/MIROC6/historical/r1i1p1f1/Omon/tos/gn/v20181212/tos_Omon_MIROC6_historical_r1i1p1f1_gn_195001-201412.nc'

## Load data from the opendap server into a dataarray

In [ ]:
ds = xr.open_dataset(opendap_data_url)

xarray have built-in functions and cartopy integration but they are limited, definitely not publication-type plots

In [ ]:
ds.sel(time='1967-06')['tos'].plot(vmin=-2, vmax=32, cmap='gist_ncar')

For that, we want to extract the data and plot it with the cartopy package. Cartopy doesn't like the discontinuity in longitude
so we're gonna remap onto a regular grid. Another option is to roll along longitude but the number of cells to roll is not constant when approching the north end and requires a find per line which is not very efficient in xaray.

### Plotting functions

In [ ]:
def plot_robinson(lon, lat, data, dict_plt):
    plt.figure(figsize=[15, 10])
    ax = plt.axes(projection=ccrs.Robinson(central_longitude=180))
    C = plt.contourf(lon, lat, data, np.arange(dict_plt['vmin'], dict_plt['vmax'], dict_plt['cstep']),
                     cmap=dict_plt['cmap'],
                     transform=ccrs.PlateCarree())
    plt.colorbar(C, shrink=0.6)
    ax.coastlines()
    ax.stock_img()
    
    # add the nino3.4 box
    plt.plot(-170 * np.ones(11), np.arange(-5,5+1),'k--', transform=ccrs.PlateCarree())
    plt.plot(-120 * np.ones(11), np.arange(-5,5+1),'k--', transform=ccrs.PlateCarree())
    plt.plot(np.arange(-170,-120+5,5), 5 * np.ones(11),'k--', transform=ccrs.PlateCarree())
    plt.plot(np.arange(-170,-120+5,5), -5 * np.ones(11),'k--', transform=ccrs.PlateCarree())

def resample(dataarray, hres=1):
    ''' resample the tripolar grid onto a regular grid for plotting purpose only'''
    # input grid
    lons_in = dataarray['longitude'].values.ravel()
    lons_in[np.where(lons_in > 180)] -= 360
    lats_in = dataarray['latitude'].values.ravel()

    grid_src = pyresample.geometry.SwathDefinition(lons=lons_in, lats=lats_in)

    # output grid
    lon = np.arange(-180, 180, hres) + hres/2
    lat = np.arange(-90, 90, hres) + hres/2
    lon_out, lat_out = np.meshgrid(lon, lat)

    grid_target = pyresample.geometry.GridDefinition(lons=lon_out,
                                                      lats=lat_out)

    data = pyresample.kd_tree.resample_nearest(grid_src, dataarray.values,
                                                grid_target,
                                                radius_of_influence=100000,
                                                fill_value=0)
    return lon, lat, data

## SST plot for June 1967

In [ ]:
lon, lat, data = resample(ds.sel(time='1967-06')['tos'].squeeze('time'), hres=1)
dict_plt= {'vmin': -2, 'vmax': 32, 'cstep': 0.5, 'cmap': cm.gist_ncar}
plot_robinson(lon, lat, data, dict_plt)

### Functions for spatial averaging with xarray

In [ ]:
def compute_area(dataset):
    ''' compute cell area.
    close to the equator the grid is mercator type so we don't need to
    correct for grid cell rotation/distortion
    
    vertices are located at the corners of the cell as follows:
    
    North
    
    3 --- 2
    |     |
 W  |     | E
    0 --- 1
    
    South
    
    
    '''
    Rearth = 6378000. #meters
    deg2rad = np.pi / 180
    
    lat_center_point = 0.5 * (dataset['vertices_latitude'].isel(vertices=1) + dataset['vertices_latitude'].isel(vertices=2))
    dlon = dataset['vertices_longitude'].isel(vertices=1) - dataset['vertices_longitude'].isel(vertices=0)
    dlat = dataset['vertices_latitude'].isel(vertices=2) - dataset['vertices_latitude'].isel(vertices=1)
    
    dx = Rearth * deg2rad * dlon * np.cos( deg2rad * lat_center_point)
    dy = Rearth * deg2rad * dlat
    
    dataset['area'] = dx * dy 
    return dataset

In [ ]:
def avg2d(dataarray, cell_area):
    ''' compute horizontal spatial average'''
    out = (dataarray * cell_area).sum(dim=['x','y']) / \
    (cell_area).where(dataarray.fillna(-9999) != -9999).sum(dim=['x','y'])
    return out

## Compute the climatology in the nino3.4 box

### monthly climatology

extract the area of interest, using where conditions. It would be more elegant if longitude/latitude where dimensions (i.e 1d). 
In this case, we could use ds['tos'].sel(longitude=slice(lonmin,lonmax),latitude=slice(latmin, latmax))

In [ ]:
nino34_box = ds['tos'].where(ds['longitude'] >= 360-170).where(ds['longitude'] <= 360-120).where(ds['latitude'] >= -5).where(ds['latitude'] <= 5)

Compute the monthly climatology: group by month, then average along time axis.

In [ ]:
clim_nino34_box = nino34_box.groupby(nino34_box.time.dt.month).mean(dim='time')

### compute the spatial average

First we need to add the area of the cells to the dataset:

In [ ]:
ds = compute_area(ds)

Then we can compute the spatial average with the function defined above to compute the climatology in the nino3.4 box

In [ ]:
clim_nino34_box_timeserie = avg2d(clim_nino34_box, ds['area'])

Plot the result:

In [ ]:
clim_nino34_box_timeserie.plot()

## Compute the anomaly

Compute the anomaly:

In [ ]:
anomaly_nino34box = nino34_box.groupby(nino34_box.time.dt.month) - clim_nino34_box

Run the spatial average:

In [ ]:
anomaly_nino34box_timeserie = avg2d(anomaly_nino34box, ds['area'])

plot:

In [ ]:
anomaly_nino34box_timeserie.plot()